<!--BOOK_INFORMATION-->
<a href="https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-opencv" target="_blank"><img align="left" src="data/cover.jpg" style="width: 76px; height: 100px; background: white; padding: 1px; border: 1px solid black; margin-right:10px;"></a>
*This notebook contains an excerpt from the book [Machine Learning for OpenCV](https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-opencv) by Michael Beyeler.
The code is released under the [MIT license](https://opensource.org/licenses/MIT),
and is available on [GitHub](https://github.com/mbeyeler/opencv-machine-learning).*

*Note that this excerpt contains only the raw code - the book is rich with additional explanations and illustrations.
If you find this content useful, please consider supporting the work by
[buying the book](https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-opencv)!*

<!--NAVIGATION-->
< [Combining Different Models Into a Voting Classifier](10.05-Combining-Different-Models-Into-a-Voting-Classifier.ipynb) | [Contents](../README.md) | [Evaluating a Model](11.01-Evaluating-a-Model.ipynb) >

# Selecting the Right Model with Hyper-Parameter Tuning

In this chapter, we will dive deeper into **model evaluation** and **hyperparameter
tuning**. Assume that we have two different models that might apply to our task. How can
we know which one is better? Answering this question often involves repeatedly fitting
different versions of our model to different subsets of the data, such as in **cross-validation**
and **bootstrapping**. In combination with different scoring functions, we can obtain reliable
estimates of the generalization performance of our models.

But what if two different models give similar results? Can we be sure that the two models
are equivalent, or is it possible that one of them just got lucky? How can we know whether
one of them is significantly better than the other? Answering these questions will lead us to
discussing some useful statistical tests such as **Students t-test** and **McNemar's test**.

As we will get familiar with these techniques, we will also want to answer the following
questions:
- What's the best strategy to tweak the hyperparameters of a model?
- How can we compare the performance of different models in a fair way?
- How do we select the right machine learning tool for the task at hand?

## Outline

- [Evaluating a Model](11.01-Evaluating-a-Model.ipynb)
- [Understanding Cross-Validation, Bootstrapping, and McNemar's Test](11.02-Understanding-Cross-Validation-Bootstrapping-and-McNemar's-Test.ipynb)
- [Tuning Hyperparameters with Grid Search](11.03-Tuning-Hyperparameters-with-Grid-Search.ipynb)
- [Chaining Algorithms Together to Form a Pipeline](11.04-Chaining-Algorithms-Together-to-Form-a-Pipeline.ipynb)
   

<!--NAVIGATION-->
< [Combining Different Models Into a Voting Classifier](10.05-Combining-Different-Models-Into-a-Voting-Classifier.ipynb) | [Contents](../README.md) | [Evaluating a Model](11.01-Evaluating-a-Model.ipynb) >